<a href="https://colab.research.google.com/github/leandroars/reel-search/blob/master/teste3d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Verificar qual GPU o Google te deu (esperamos uma Tesla T4 ou superior)
!nvidia-smi

# 2. Clonar o repositório oficial
!git clone https://github.com/facebookresearch/sam-3d-objects.git

# 3. Entrar na pasta do projeto
%cd sam-3d-objects

Tue Nov 25 03:08:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!huggingface-cli whoami

⚠️  Warning: 'huggingface-cli whoami' is deprecated. Use 'hf auth whoami' instead.
leandroars
orgs:  MakerBros


In [4]:
# 1. Garantir que a ferramenta de download está na versão certa
!pip install "huggingface-hub[cli]<1.0"

# 2. Criar a pasta onde o modelo vai ficar
!mkdir -p checkpoints/hf

print("--- ⏳ Iniciando o download do Modelo... aguarde... ---")

# 3. Baixar os arquivos do Facebook
!export TAG=hf && \
 huggingface-cli download \
 --repo-type model \
 --local-dir checkpoints/$TAG-download \
 --max-workers 4 \
 facebook/sam-3d-objects

# 4. Organizar os arquivos na pasta correta
!mv checkpoints/hf-download/checkpoints/* checkpoints/hf/
!rm -rf checkpoints/hf-download

print("--- ✅ Sucesso! Modelo baixado e pronto. ---")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 3.3 MB/s eta 0:00:00
--- ⏳ Iniciando o download do Modelo... aguarde... ---
⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Fetching 28 files:   0% 0/28 [00:00<?, ?it/s]
LICENSE: 100% 8.20k/8.20k [00:00<00:00, 48.0MB/s]
Download complete. Moving file to checkpoints/hf-download/LICENSE
README.md: 100% 7.20k/7.20k [00:00<00:00, 23.9MB/s]
Download complete. Moving file to checkpoints/hf-download/README.md
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_http.py", line 402, in hf_raise_for_status
    response.raise_for_status()
  File "/usr/local/lib/python3.12/dist-packages/requests/models.py", line 1026, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 403 Client Error: Forbidden for url: https://huggingface.co/facebook/sam-3d-objects/resolve/a878e8e0f869d6db637cf2f096ff5ae5fac786b6/.gitatt

In [9]:
import sys
import os
import torch
import numpy as np
from PIL import Image

# 1. FORÇAR a entrada na pasta correta
%cd /content/sam-3d-objects

# 2. Adicionar o caminho EXATO onde está o arquivo inference.py
sys.path.append("/content/sam-3d-objects/notebook")

# 3. Agora tentamos importar novamente
from inference import Inference, load_image

print("--- ⚙️ Carregando o modelo (pode levar 1 minuto)... ---")
tag = "hf"
config_path = f"checkpoints/{tag}/pipeline.yaml"

# Inicializa a IA
inference = Inference(config_path, compile=False)
print("--- 🚀 Modelo carregado e pronto! ---")

[Errno 2] No such file or directory: '/content/sam-3d-objects'
/content


ModuleNotFoundError: No module named 'inference'

In [10]:
import os
import sys

# 1. Verifica se a pasta sumiu e baixa de novo se precisar
if not os.path.exists('/content/sam-3d-objects'):
    print("⚠️ A pasta sumiu! O Colab deve ter reiniciado.")
    print("--- Baixando o código novamente... ---")
    %cd /content/
    !git clone https://github.com/facebookresearch/sam-3d-objects.git
else:
    print("✅ A pasta ainda está aqui.")

# 2. Entra na pasta correta
%cd /content/sam-3d-objects

# 3. Adiciona o caminho ao sistema (para o Python achar o arquivo)
sys.path.append("/content/sam-3d-objects/notebook")

# 4. Tenta importar novamente
try:
    from inference import Inference, load_image
    print("\n🎉 Sucesso! O módulo 'inference' foi encontrado.")
except ImportError:
    print("\n❌ Ainda deu erro. Se isso acontecer, você precisa subir lá no topo e rodar as células de INSTALAÇÃO (!pip install...) novamente.")

⚠️ A pasta sumiu! O Colab deve ter reiniciado.
--- Baixando o código novamente... ---
/content
Cloning into 'sam-3d-objects'...
remote: Enumerating objects: 1101, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 1101 (delta 93), reused 82 (delta 82), pack-reused 985 (from 1)
Receiving objects: 100% (1101/1101), 266.74 MiB | 24.13 MiB/s, done.
Resolving deltas: 100% (401/401), done.
Updating files: 100% (415/415), done.
/content/sam-3d-objects

❌ Ainda deu erro. Se isso acontecer, você precisa subir lá no topo e rodar as células de INSTALAÇÃO (!pip install...) novamente.


In [11]:
# Entrar na pasta certa (garantia)
%cd /content/sam-3d-objects

print("--- 🔄 Reinstalando dependências (Pode demorar um pouco)... ---")
!pip install -e ".[dev]"
!pip install -e ".[p3d]"
!pip install -e ".[inference]"
!pip install rembg

print("--- 🔧 Aplicando correção do sistema ---")
!chmod +x patching/hydra
!./patching/hydra

print("✅ Instalação concluída!")

/content/sam-3d-objects
--- 🔄 Reinstalando dependências (Pode demorar um pouco)... ---
Obtaining file:///content/sam-3d-objects
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/microsoft/MoGe.git (to revision a8c37341bc0325ca99b9d57981cc3bb2bd3e255b) to /tmp/pip-install-xt80iz_v/moge_6dc0a244373142c091a37497021fcd4b
  Running command git clone --filter=blob:none --quiet https://github.com/microsoft/MoGe.git /tmp/pip-install-xt80iz_v/moge_6dc0a244373142c091a37497021fcd4b
  Running command git rev-parse -q --verify 'sha^a8c37341bc0325ca99b9d57981cc3bb2bd3e255b'
  Running command git fetch -q https://github.com/microsoft/MoGe.git a8c37341bc0325ca99b9d57981cc3bb2bd3e255b
  Running command git checkout -q a8c37341bc0325ca99b9d57981cc3bb2bd3e255b
  Resolved https:/

In [12]:
# 1. Instalar o Hydra manualmente (forçar a instalação)
!pip install hydra-core --upgrade

# 2. Agora sim, aplicar a correção (o patch)
print("--- Tentando aplicar o patch novamente... ---")
!chmod +x patching/hydra
!./patching/hydra

print("✅ Agora deve ter funcionado!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 6.1 MB/s eta 0:00:00
--- Tentando aplicar o patch novamente... ---
✅ Agora deve ter funcionado!


In [13]:
# 1. Instalar o Hydra manualmente (forçar a instalação)
!pip install hydra-core --upgrade

# 2. Agora sim, aplicar a correção (o patch)
print("--- Tentando aplicar o patch novamente... ---")
!chmod +x patching/hydra
!./patching/hydra

print("✅ Agora deve ter funcionado!")

--- Tentando aplicar o patch novamente... ---
✅ Agora deve ter funcionado!


In [14]:
from huggingface_hub import login
login()

In [15]:
# Recriar a pasta e baixar o modelo
!mkdir -p checkpoints/hf
!pip install "huggingface-hub[cli]<1.0"

print("--- ⬇️ Baixando o modelo... (Aguarde terminar) ---")
!export TAG=hf && \
 huggingface-cli download \
 --repo-type model \
 --local-dir checkpoints/$TAG-download \
 --max-workers 4 \
 facebook/sam-3d-objects

!mv checkpoints/hf-download/checkpoints/* checkpoints/hf/
!rm -rf checkpoints/hf-download

print("✅ Modelo baixado com sucesso!")

--- ⬇️ Baixando o modelo... (Aguarde terminar) ---
⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Fetching 28 files:   0% 0/28 [00:00<?, ?it/s]Still waiting to acquire lock on checkpoints/hf-download/.cache/huggingface/.gitignore.lock (elapsed: 0.1 seconds)
Fetching 28 files:   0% 0/28 [00:00<?, ?it/s]
README.md: 100% 7.20k/7.20k [00:00<00:00, 43.5MB/s]
Download complete. Moving file to checkpoints/hf-download/README.md
LICENSE: 100% 8.20k/8.20k [00:00<00:00, 54.5MB/s]
Download complete. Moving file to checkpoints/hf-download/LICENSE
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_http.py", line 402, in hf_raise_for_status
    response.raise_for_status()
  File "/usr/local/lib/python3.12/dist-packages/requests/models.py", line 1026, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 403 Client Error: Forbidden for url: https://huggingface.co/fa

In [17]:
import sys
import torch
import numpy as np
from PIL import Image
from google.colab import files
import matplotlib.pyplot as plt
from rembg import remove

# Configurar caminhos
sys.path.append("/content/sam-3d-objects/notebook")
%cd /content/sam-3d-objects
from inference import Inference, load_image

print("--- ⚙️ Carregando a IA... ---")
tag = "hf"
config_path = f"checkpoints/{tag}/pipeline.yaml"
inference = Inference(config_path, compile=False)

# 1. Upload da imagem
print("\n👇 ENVIE SUA FOTO AGORA:")
uploaded = files.upload()
filename = next(iter(uploaded))

# 2. Processar
image_pil = Image.open(filename).convert("RGB")
print("--- ✂️ Criando máscara... ---")
mask_pil = remove(image_pil, only_mask=True).convert("L").resize(image_pil.size)

# 3. Gerar 3D
print("--- 🎲 Criando objeto 3D... ---")
output = inference(image_pil, mask_pil, seed=42)
output["gs"].save_ply("resultado_3d.ply")

print("\n✅ PRONTO! Seu arquivo 'resultado_3d.ply' está na pasta à esquerda.")

ModuleNotFoundError: No module named 'onnxruntime'

In [18]:
# 1. Correção rápida: Instalar a peça que falta
print("--- 🔧 Instalando onnxruntime... ---")
!pip install onnxruntime onnxruntime-gpu rembg

import sys
import torch
import numpy as np
from PIL import Image
from google.colab import files
import matplotlib.pyplot as plt
from rembg import remove

# Configurar caminhos
sys.path.append("/content/sam-3d-objects/notebook")
%cd /content/sam-3d-objects
from inference import Inference, load_image

# 2. Carregar a IA
print("--- ⚙️ Carregando a IA... ---")
tag = "hf"
config_path = f"checkpoints/{tag}/pipeline.yaml"
inference = Inference(config_path, compile=False)

# 3. Upload da imagem
print("\n👇 ENVIE SUA FOTO AGORA:")
uploaded = files.upload()
filename = next(iter(uploaded))

# 4. Processar
image_pil = Image.open(filename).convert("RGB")
print("--- ✂️ Criando máscara... ---")
mask_pil = remove(image_pil, only_mask=True).convert("L").resize(image_pil.size)

# Mostrar a máscara para você ver se ficou boa
plt.imshow(mask_pil, cmap='gray')
plt.title("Máscara gerada")
plt.axis('off')
plt.show()

# 5. Gerar 3D
print("--- 🎲 Criando objeto 3D... (Isso leva uns 60 segundos) ---")
output = inference(image_pil, mask_pil, seed=42)
output["gs"].save_ply("resultado_3d.ply")

print("\n✅ PRONTO! Seu arquivo 'resultado_3d.ply' está na pasta à esquerda.")

--- 🔧 Instalando onnxruntime... ---
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.3 MB/s eta 0:00:00
/content/sam-3d-objects


ModuleNotFoundError: No module named 'inference'

In [19]:
import sys
import torch
import numpy as np
from PIL import Image
from google.colab import files
import matplotlib.pyplot as plt
from rembg import remove

# 1. Configurar onde o Python deve buscar os arquivos
sys.path.append("/content/sam-3d-objects/notebook")
%cd /content/sam-3d-objects
from inference import Inference, load_image

# 2. Carregar o cérebro da IA (Modelos)
print("--- ⚙️ Carregando o modelo SAM 3D... ---")
tag = "hf"
config_path = f"checkpoints/{tag}/pipeline.yaml"
inference = Inference(config_path, compile=False)

# 3. Pedir a sua foto
print("\n👇 CLIQUE NO BOTÃO ABAIXO PARA ENVIAR SUA FOTO:")
uploaded = files.upload()
filename = next(iter(uploaded))

# 4. Preparar a imagem (remover fundo automaticamente)
print("--- ✂️ Criando máscara do objeto... ---")
image_pil = Image.open(filename).convert("RGB")
mask_pil = remove(image_pil, only_mask=True).convert("L").resize(image_pil.size)

# Mostrar o que será processado
plt.figure(figsize=(8,4))
plt.subplot(1,2,1); plt.imshow(image_pil); plt.title("Original")
plt.subplot(1,2,2); plt.imshow(mask_pil, cmap='gray'); plt.title("Máscara")
plt.axis('off')
plt.show()

# 5. Gerar o arquivo 3D
print("--- 🎲 Criando o 3D... (Aguarde ~1 minuto) ---")
output = inference(image_pil, mask_pil, seed=42)

# Salvar
output_name = "meu_modelo_3d.ply"
output["gs"].save_ply(output_name)

print(f"\n✅ SUCESSO! O arquivo '{output_name}' foi salvo.")
print("👉 Procure na pasta à esquerda (ícone de pasta) e faça o download!")

/content/sam-3d-objects


ModuleNotFoundError: No module named 'inference'

In [21]:
import os
import sys
import shutil
import torch
from PIL import Image
from google.colab import files
import matplotlib.pyplot as plt
from rembg import remove

print("--- 🕵️‍♂️ Iniciando diagnóstico e correção de pastas... ---")

# 1. Verificar se a pasta do projeto existe. Se não, baixa de novo.
if not os.path.exists("/content/sam-3d-objects"):
    print("⚠️ A pasta do projeto sumiu. Baixando novamente...")
    !git clone https://github.com/facebookresearch/sam-3d-objects.git
else:
    print("✅ Pasta do projeto encontrada.")

# 2. TRUQUE: Copiar o arquivo 'inference.py' para a raiz para facilitar
source_file = "/content/sam-3d-objects/notebook/inference.py"
dest_file = "/content/inference.py"

if os.path.exists(source_file):
    shutil.copy(source_file, dest_file)
    print("✅ Arquivo 'inference.py' movido para o local correto.")
else:
    # Se o arquivo não existe, algo grave aconteceu com o download
    print("❌ ERRO: O arquivo inference.py não foi encontrado. O download do Github falhou.")
    # Tenta baixar apenas o arquivo necessário
    !wget https://raw.githubusercontent.com/facebookresearch/sam-3d-objects/main/notebook/inference.py -O /content/inference.py

# 3. Importar (agora vai funcionar porque o arquivo está ao lado)
sys.path.append("/content")
try:
    from inference import Inference, load_image
    print("🎉 Sucesso! O módulo 'inference' foi carregado.")
except ImportError as e:
    print(f"❌ Ainda deu erro no import: {e}")

# ---------------------------------------------------------
# CÓDIGO DE GERAÇÃO 3D
# ---------------------------------------------------------

# 4. Carregar a IA
print("\n--- ⚙️ Carregando o modelo SAM 3D... ---")
# Caminho absoluto para evitar erros
config_path = "/content/sam-3d-objects/checkpoints/hf/pipeline.yaml"

if not os.path.exists(config_path):
    print(f"❌ Erro: Não achei o arquivo de configuração em: {config_path}")
    print("Você baixou o modelo? (Passo do Login e Download)")
else:
    inference = Inference(config_path, compile=False)

    # 5. Upload da imagem
    print("\n👇 CLIQUE NO BOTÃO ABAIXO PARA ENVIAR SUA FOTO:")
    uploaded = files.upload()
    if not uploaded:
        print("Nenhuma imagem enviada.")
    else:
        filename = next(iter(uploaded))

        # 6. Processar
        print("--- ✂️ Criando máscara do objeto... ---")
        image_pil = Image.open(filename).convert("RGB")
        mask_pil = remove(image_pil, only_mask=True).convert("L").resize(image_pil.size)

        # Mostrar
        plt.figure(figsize=(8,4))
        plt.subplot(1,2,1); plt.imshow(image_pil); plt.title("Original")
        plt.subplot(1,2,2); plt.imshow(mask_pil, cmap='gray'); plt.title("Máscara")
        plt.axis('off')
        plt.show()

        # 7. Gerar 3D
        print("--- 🎲 Criando o 3D... (Aguarde ~1 minuto) ---")
        output = inference(image_pil, mask_pil, seed=42)

        output_name = "meu_modelo_3d.ply"
        output["gs"].save_ply(output_name)

        print(f"\n✅ SUCESSO FINAL! O arquivo '{output_name}' foi salvo na pasta à esquerda.")

--- 🕵️‍♂️ Iniciando diagnóstico e correção de pastas... ---
✅ Pasta do projeto encontrada.
✅ Arquivo 'inference.py' movido para o local correto.


KeyError: 'CONDA_PREFIX'

In [22]:
import os
import sys
import shutil
import torch
from PIL import Image
from google.colab import files
import matplotlib.pyplot as plt
from rembg import remove

print("--- 🕵️‍♂️ Iniciando diagnóstico e correção de pastas... ---")

# 1. Verificar se a pasta do projeto existe. Se não, baixa de novo.
if not os.path.exists("/content/sam-3d-objects"):
    print("⚠️ A pasta do projeto sumiu. Baixando novamente...")
    !git clone https://github.com/facebookresearch/sam-3d-objects.git
else:
    print("✅ Pasta do projeto encontrada.")

# 2. TRUQUE: Copiar o arquivo 'inference.py' para a raiz para facilitar
source_file = "/content/sam-3d-objects/notebook/inference.py"
dest_file = "/content/inference.py"

if os.path.exists(source_file):
    shutil.copy(source_file, dest_file)
    print("✅ Arquivo 'inference.py' movido para o local correto.")
else:
    # Se o arquivo não existe, algo grave aconteceu com o download
    print("❌ ERRO: O arquivo inference.py não foi encontrado. O download do Github falhou.")
    # Tenta baixar apenas o arquivo necessário
    !wget https://raw.githubusercontent.com/facebookresearch/sam-3d-objects/main/notebook/inference.py -O /content/inference.py

# 3. Importar (agora vai funcionar porque o arquivo está ao lado)
sys.path.append("/content")
try:
    from inference import Inference, load_image
    print("🎉 Sucesso! O módulo 'inference' foi carregado.")
except ImportError as e:
    print(f"❌ Ainda deu erro no import: {e}")

# ---------------------------------------------------------
# CÓDIGO DE GERAÇÃO 3D
# ---------------------------------------------------------

# 4. Carregar a IA
print("\n--- ⚙️ Carregando o modelo SAM 3D... ---")
# Caminho absoluto para evitar erros
config_path = "/content/sam-3d-objects/checkpoints/hf/pipeline.yaml"

if not os.path.exists(config_path):
    print(f"❌ Erro: Não achei o arquivo de configuração em: {config_path}")
    print("Você baixou o modelo? (Passo do Login e Download)")
else:
    inference = Inference(config_path, compile=False)

    # 5. Upload da imagem
    print("\n👇 CLIQUE NO BOTÃO ABAIXO PARA ENVIAR SUA FOTO:")
    uploaded = files.upload()
    if not uploaded:
        print("Nenhuma imagem enviada.")
    else:
        filename = next(iter(uploaded))

        # 6. Processar
        print("--- ✂️ Criando máscara do objeto... ---")
        image_pil = Image.open(filename).convert("RGB")
        mask_pil = remove(image_pil, only_mask=True).convert("L").resize(image_pil.size)

        # Mostrar
        plt.figure(figsize=(8,4))
        plt.subplot(1,2,1); plt.imshow(image_pil); plt.title("Original")
        plt.subplot(1,2,2); plt.imshow(mask_pil, cmap='gray'); plt.title("Máscara")
        plt.axis('off')
        plt.show()

        # 7. Gerar 3D
        print("--- 🎲 Criando o 3D... (Aguarde ~1 minuto) ---")
        output = inference(image_pil, mask_pil, seed=42)

        output_name = "meu_modelo_3d.ply"
        output["gs"].save_ply(output_name)

        print(f"\n✅ SUCESSO FINAL! O arquivo '{output_name}' foi salvo na pasta à esquerda.")

--- 🕵️‍♂️ Iniciando diagnóstico e correção de pastas... ---
✅ Pasta do projeto encontrada.
✅ Arquivo 'inference.py' movido para o local correto.


KeyError: 'CONDA_PREFIX'

In [23]:
import os
import sys
import shutil
import torch
from PIL import Image
from google.colab import files
import matplotlib.pyplot as plt
from rembg import remove

# --- CORREÇÃO DO ERRO ---
# Enganar o código dizendo que o Conda está na pasta padrão do Colab
os.environ['CONDA_PREFIX'] = '/usr/local'
# ------------------------

print("--- 🕵️‍♂️ Iniciando... ---")

# 1. Verificar pasta do projeto
if not os.path.exists("/content/sam-3d-objects"):
    print("⚠️ Pasta sumiu. Baixando de novo...")
    !git clone https://github.com/facebookresearch/sam-3d-objects.git

# 2. Copiar arquivo inference.py para a raiz (evita erro de import)
source_file = "/content/sam-3d-objects/notebook/inference.py"
dest_file = "/content/inference.py"
if os.path.exists(source_file):
    shutil.copy(source_file, dest_file)

# 3. Importar
sys.path.append("/content")
try:
    from inference import Inference, load_image
    print("✅ Sucesso! Módulo carregado.")
except Exception as e:
    print(f"❌ Erro ainda: {e}")

# 4. Carregar a IA
print("\n--- ⚙️ Carregando o modelo SAM 3D... ---")
config_path = "/content/sam-3d-objects/checkpoints/hf/pipeline.yaml"

if os.path.exists(config_path):
    inference = Inference(config_path, compile=False)

    # 5. Upload da imagem
    print("\n👇 CLIQUE NO BOTÃO ABAIXO PARA ENVIAR SUA FOTO:")
    uploaded = files.upload()

    if uploaded:
        filename = next(iter(uploaded))

        # 6. Processar Máscara
        print("--- ✂️ Criando máscara... ---")
        image_pil = Image.open(filename).convert("RGB")
        mask_pil = remove(image_pil, only_mask=True).convert("L").resize(image_pil.size)

        # Mostrar preview
        plt.figure(figsize=(8,4))
        plt.subplot(1,2,1); plt.imshow(image_pil); plt.title("Original")
        plt.subplot(1,2,2); plt.imshow(mask_pil, cmap='gray'); plt.title("Máscara")
        plt.axis('off'); plt.show()

        # 7. Gerar 3D
        print("--- 🎲 Criando o 3D... (Aguarde ~1 minuto) ---")
        output = inference(image_pil, mask_pil, seed=42)

        output_name = "meu_modelo_3d.ply"
        output["gs"].save_ply(output_name)

        print(f"\n✅ SUCESSO! Baixe o arquivo '{output_name}' na pasta à esquerda.")
    else:
        print("Nenhuma imagem selecionada.")
else:
    print("❌ Erro: O arquivo pipeline.yaml não foi encontrado. O download do modelo (passo anterior) falhou?")

--- 🕵️‍♂️ Iniciando... ---
❌ Erro ainda: No module named 'utils3d'

--- ⚙️ Carregando o modelo SAM 3D... ---
❌ Erro: O arquivo pipeline.yaml não foi encontrado. O download do modelo (passo anterior) falhou?


In [24]:
# Célula de Reparo e Download
import os

!mkdir -p checkpoints/hf
!pip install "huggingface-hub[cli]<1.0"

print("--- ⬇️ BAIXANDO O MODELO (Não feche a aba!) ---")
# Baixa apenas se não existir
if not os.path.exists("checkpoints/hf/pipeline.yaml"):
    !export TAG=hf && \
     huggingface-cli download \
     --repo-type model \
     --local-dir checkpoints/$TAG-download \
     --max-workers 4 \
     facebook/sam-3d-objects

    !mv checkpoints/hf-download/checkpoints/* checkpoints/hf/
    !rm -rf checkpoints/hf-download
    print("✅ Download Concluído!")
else:
    print("✅ O modelo já está aí! Pode pular para a próxima etapa.")

--- ⬇️ BAIXANDO O MODELO (Não feche a aba!) ---
⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Fetching 28 files:   0% 0/28 [00:00<?, ?it/s]
LICENSE: 100% 8.20k/8.20k [00:00<00:00, 42.6MB/s]
Download complete. Moving file to checkpoints/hf-download/LICENSE
README.md: 100% 7.20k/7.20k [00:00<00:00, 48.7MB/s]
Download complete. Moving file to checkpoints/hf-download/README.md
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_http.py", line 402, in hf_raise_for_status
    response.raise_for_status()
  File "/usr/local/lib/python3.12/dist-packages/requests/models.py", line 1026, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 403 Client Error: Forbidden for url: https://huggingface.co/facebook/sam-3d-objects/resolve/a878e8e0f869d6db637cf2f096ff5ae5fac786b6/.gitattributes

The above exception was the direct cause of the following exception:

Traceb

In [25]:
import sys
import os
import torch
import numpy as np
from PIL import Image
from google.colab import files
import matplotlib.pyplot as plt
from rembg import remove

# --- CORREÇÃO DOS ERROS ---
# 1. Define a variável que faltava (Conda)
os.environ['CONDA_PREFIX'] = '/usr/local'

# 2. Aponta para a pasta correta ONDE ESTÃO os arquivos (não movemos nada)
sys.path.append("/content/sam-3d-objects/notebook")

# 3. Entra na pasta raiz do projeto para o modelo achar os pesos
%cd /content/sam-3d-objects

# --------------------------

try:
    from inference import Inference, load_image
    print("✅ Módulo Inference carregado corretamente!")
except ImportError as e:
    print(f"❌ Erro crítico: {e}")
    print("Verifique se a pasta '/content/sam-3d-objects/notebook' existe na aba de arquivos.")

# Verificação final do modelo
config_path = "checkpoints/hf/pipeline.yaml"
if not os.path.exists(config_path):
    print("\n🚨 PARE TUDO! O arquivo pipeline.yaml ainda não existe.")
    print("Você precisa rodar a ETAPA 1 acima para baixar o modelo.")
else:
    print("\n--- ⚙️ Carregando a IA... ---")
    inference = Inference(config_path, compile=False)

    print("\n👇 ENVIE SUA FOTO AGORA:")
    uploaded = files.upload()

    if uploaded:
        filename = next(iter(uploaded))
        image_pil = Image.open(filename).convert("RGB")

        print("--- ✂️ Processando imagem... ---")
        mask_pil = remove(image_pil, only_mask=True).convert("L").resize(image_pil.size)

        print("--- 🎲 Gerando 3D... ---")
        output = inference(image_pil, mask_pil, seed=42)

        output["gs"].save_ply("resultado_final.ply")
        print("\n🏆 SUCESSO! Baixe 'resultado_final.ply' na pasta do projeto.")
    else:
        print("Nenhuma imagem enviada.")

/content/sam-3d-objects
❌ Erro crítico: No module named 'utils3d'
Verifique se a pasta '/content/sam-3d-objects/notebook' existe na aba de arquivos.

🚨 PARE TUDO! O arquivo pipeline.yaml ainda não existe.
Você precisa rodar a ETAPA 1 acima para baixar o modelo.


In [26]:
import os
import sys
import shutil

print("--- 🧹 FAXINA: Removendo arquivos conflitantes... ---")
# 1. Apagar o arquivo que está confundindo o Python
if os.path.exists("/content/inference.py"):
    os.remove("/content/inference.py")
    print("✅ Arquivo 'inference.py' antigo removido.")

# 2. Resetar caminhos do Python
if "/content" in sys.path:
    sys.path.remove("/content")

print("--- 🔍 Verificando Modelo... ---")
# 3. Garantir que o modelo existe (Download obrigatório)
# Se o arquivo não estiver lá, ele baixa.
config_path = "/content/sam-3d-objects/checkpoints/hf/pipeline.yaml"

if not os.path.exists(config_path):
    print("⚠️ O modelo não foi encontrado. Iniciando download (pode demorar 5 min)...")
    # Instalar CLI do HuggingFace
    !pip install "huggingface-hub[cli]<1.0" > /dev/null

    # Criar pastas
    !mkdir -p /content/sam-3d-objects/checkpoints/hf

    # Baixar
    !export TAG=hf && \
     huggingface-cli download \
     --repo-type model \
     --local-dir /content/sam-3d-objects/checkpoints/$TAG-download \
     --max-workers 4 \
     facebook/sam-3d-objects

    # Mover
    !mv /content/sam-3d-objects/checkpoints/hf-download/checkpoints/* /content/sam-3d-objects/checkpoints/hf/
    !rm -rf /content/sam-3d-objects/checkpoints/hf-download
    print("✅ Download Concluído!")
else:
    print("✅ O modelo já existe! Pulando download.")

# ---------------------------------------------------------
# EXECUÇÃO DO SAM 3D
# ---------------------------------------------------------
print("--- 🚀 Iniciando SAM 3D... ---")

import torch
import numpy as np
from PIL import Image
from google.colab import files
import matplotlib.pyplot as plt
from rembg import remove

# Configurar ambiente
os.environ['CONDA_PREFIX'] = '/usr/local'
sys.path.append("/content/sam-3d-objects/notebook")
%cd /content/sam-3d-objects

# Importar do local correto
try:
    # Forçar recarregamento para ignorar cache antigo
    if 'inference' in sys.modules:
        del sys.modules['inference']
    from inference import Inference, load_image
    print("✅ Sistema carregado corretamente.")
except ImportError as e:
    print(f"❌ Erro fatal no import: {e}")
    print("Dica: Se o erro persistir, vá em 'Runtime' > 'Restart Session' e rode APENAS as instalações iniciais e este bloco.")

# Carregar IA
inference = Inference(config_path, compile=False)

# Upload e Geração
print("\n👇 ENVIE SUA FOTO AGORA:")
uploaded = files.upload()

if uploaded:
    filename = next(iter(uploaded))
    image_pil = Image.open(filename).convert("RGB")

    print("--- ✂️ Removendo fundo... ---")
    mask_pil = remove(image_

SyntaxError: incomplete input (ipython-input-3231668985.py, line 78)

In [27]:
import sys
import os
import torch
import numpy as np
from PIL import Image
from google.colab import files
import matplotlib.pyplot as plt
from rembg import remove

# 1. Configurar ambiente e caminhos
os.environ['CONDA_PREFIX'] = '/usr/local'
sys.path.append("/content/sam-3d-objects/notebook")
os.chdir("/content/sam-3d-objects")

try:
    # Remove importação antiga da memória para evitar conflitos
    if 'inference' in sys.modules:
        del sys.modules['inference']
    from inference import Inference, load_image
    print("✅ Sistema carregado!")
except ImportError:
    print("⚠️ Atenção: Se der erro aqui, certifique-se que a pasta sam-3d-objects existe.")

# 2. Carregar a IA
print("--- ⚙️ Carregando Cérebro da IA... ---")
config_path = "/content/sam-3d-objects/checkpoints/hf/pipeline.yaml"

if os.path.exists(config_path):
    inference = Inference(config_path, compile=False)

    # 3. Upload e Geração
    print("\n👇 CLIQUE NO BOTÃO ABAIXO E ENVIE SUA FOTO:")
    uploaded = files.upload()

    if uploaded:
        filename = next(iter(uploaded))
        image_pil = Image.open(filename).convert("RGB")

        print("--- ✂️ Removendo fundo da imagem... ---")
        # A LINHA QUE TINHA DADO ERRO AGORA ESTÁ COMPLETA ABAIXO:
        mask_pil = remove(image_pil, only_mask=True).convert("L").resize(image_pil.size)

        # Mostra o que vai ser feito
        plt.figure(figsize=(6,3))
        plt.subplot(1,2,1); plt.imshow(image_pil); plt.title("Sua Foto")
        plt.subplot(1,2,2); plt.imshow(mask_pil, cmap='gray'); plt.title("Máscara Automática")
        plt.axis('off'); plt.show()

        print("--- 🎲 Criando o 3D... (Aguarde ~1 minuto) ---")
        output = inference(image_pil, mask_pil, seed=42)

        output["gs"].save_ply("meu_3d_final.ply")
        print("\n🏆 SUCESSO! O arquivo 'meu_3d_final.ply' apareceu na pasta à esquerda. Pode baixar!")
    else:
        print("❌ Nenhuma imagem enviada.")
else:
    print("❌ ERRO: O arquivo do modelo (pipeline.yaml) não foi encontrado.")
    print("Você precisa rodar o bloco de DOWNLOAD novamente (o passo anterior).")

⚠️ Atenção: Se der erro aqui, certifique-se que a pasta sam-3d-objects existe.
--- ⚙️ Carregando Cérebro da IA... ---
❌ ERRO: O arquivo do modelo (pipeline.yaml) não foi encontrado.
Você precisa rodar o bloco de DOWNLOAD novamente (o passo anterior).


In [28]:
import os

print("--- ⬇️ INICIANDO DOWNLOAD DO MODELO (Isso é obrigatório) ---")
print("Aguarde... pode levar uns 5 a 10 minutos.")

# 1. Instalar ferramenta
!pip install "huggingface-hub[cli]<1.0" > /dev/null

# 2. Criar pastas certinhas
!mkdir -p /content/sam-3d-objects/checkpoints/hf

# 3. Baixar
!export TAG=hf && \
 huggingface-cli download \
 --repo-type model \
 --local-dir /content/sam-3d-objects/checkpoints/$TAG-download \
 --max-workers 4 \
 facebook/sam-3d-objects

# 4. Mover para o lugar final
!mv /content/sam-3d-objects/checkpoints/hf-download/checkpoints/* /content/sam-3d-objects/checkpoints/hf/
!rm -rf /content/sam-3d-objects/checkpoints/hf-download

# 5. Verificação final
if os.path.exists("/content/sam-3d-objects/checkpoints/hf/pipeline.yaml"):
    print("\n✅ SUCESSO! O arquivo 'pipeline.yaml' está no lugar certo.")
    print("👉 AGORA SIM: Pode rodar o código de gerar o 3D (Etapa 2).")
else:
    print("\n❌ ERRO: O download falhou. Tente rodar o comando 'login()' novamente antes de baixar.")

--- ⬇️ INICIANDO DOWNLOAD DO MODELO (Isso é obrigatório) ---
Aguarde... pode levar uns 5 a 10 minutos.
⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Fetching 28 files:   0% 0/28 [00:00<?, ?it/s]
LICENSE: 100% 8.20k/8.20k [00:00<00:00, 50.3MB/s]
Download complete. Moving file to /content/sam-3d-objects/checkpoints/hf-download/LICENSE
README.md: 100% 7.20k/7.20k [00:00<00:00, 45.9MB/s]
Download complete. Moving file to /content/sam-3d-objects/checkpoints/hf-download/README.md
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_http.py", line 402, in hf_raise_for_status
    response.raise_for_status()
  File "/usr/local/lib/python3.12/dist-packages/requests/models.py", line 1026, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 403 Client Error: Forbidden for url: https://huggingface.co/facebook/sam-3d-objects/resolve/a878e8e0f869d6db637cf2f096ff5a

In [30]:
from huggingface_hub import login
# Vai pedir o token. Cole o token (começa com hf_) e dê Enter.
login()

In [31]:
import os
# Código de download do modelo
!pip install "huggingface-hub[cli]<1.0" > /dev/null
!mkdir -p /content/sam-3d-objects/checkpoints/hf

print("--- ⬇️ TENTANDO BAIXAR DE NOVO... ---")
!export TAG=hf && \
 huggingface-cli download \
 --repo-type model \
 --local-dir /content/sam-3d-objects/checkpoints/$TAG-download \
 --max-workers 4 \
 facebook/sam-3d-objects

!mv /content/sam-3d-objects/checkpoints/hf-download/checkpoints/* /content/sam-3d-objects/checkpoints/hf/
!rm -rf /content/sam-3d-objects/checkpoints/hf-download

print("✅ Se não deu erro vermelho, funcionou!")

--- ⬇️ TENTANDO BAIXAR DE NOVO... ---
⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Fetching 28 files:   0% 0/28 [00:00<?, ?it/s]Downloading 'LICENSE' to '/content/sam-3d-objects/checkpoints/hf-download/.cache/huggingface/download/DhCjcNQuMpl4FL346qr3tvNUCgY=.e58d758cc3d92e6d1a57b52f7e4d203d02a67e98.incomplete'
Fetching 28 files:   0% 0/28 [00:00<?, ?it/s]
LICENSE: 100% 8.20k/8.20k [00:00<00:00, 40.2MB/s]
Download complete. Moving file to /content/sam-3d-objects/checkpoints/hf-download/LICENSE
README.md: 100% 7.20k/7.20k [00:00<00:00, 34.9MB/s]
Download complete. Moving file to /content/sam-3d-objects/checkpoints/hf-download/README.md
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_http.py", line 402, in hf_raise_for_status
    response.raise_for_status()
  File "/usr/local/lib/python3.12/dist-packages/requests/models.py", line 1026, in raise_for_status
    raise HTTPError(http_error_msg, resp

In [ ]:
print("--- Instalando dependências básicas (Dev) ---")
!pip install -e ".[dev]"

In [ ]:
print("--- Instalando PyTorch3D e Kaolin (Pode demorar alguns minutos) ---")
!pip install -e ".[p3d]"

In [ ]:
print("--- Instalando dependências de Inferência ---")
!pip install -e ".[inference]"

In [ ]:
print("---
--- Aplicando Patch ---
---")
!chmod +x patching/hydra
!./patching/hydra